In [2]:
# Reference taken from http://syspedia.de/EgoNetworks/Ego-centric%20graphs.pdf

from operator import itemgetter

import matplotlib.pyplot as plt
import networkx as nx
import random
import config
from itertools import chain, combinations
from statistics import mean
from network import modify_reciprocity, get_network_stats, network_to_csv
import pickle

# An ego-centric approach to synthesize more realistic social networks

Author: Hans-Peter Stricker

In [3]:

def clamp(val, lower, upper):
    if val <= lower:
        val = lower
    elif val > upper:
        val = upper
    return val


def get_largest_hub(G):
    node_and_degree = G.degree()
    (largest_hub, degree) = sorted(node_and_degree, key=itemgetter(1))[-1]
    return largest_hub


def powerset_of_2(iterable):
    "powerset_of_2([(),(1),(1,2),(1,2,3)]) --> [(1,2)]"
    s = list(iterable)
    powerset = chain.from_iterable(combinations(s, r) for r in range(len(s)+1))
    powerset_of_2 = [s for s in powerset if len(s) == 2]
    return powerset_of_2


def get_unique_edges(edges):
    #  return list(set([(u, v) if u <= v else (v, u) for u, v in edges]))
    return list(set([(u, v) if u <= v else (v, u) for u, v in edges]))


def get_possible_edges(e0, e1, ego_net: nx.Graph, new_nodes: list[int]):
    '''
    These then have to be linked at random with each other and with those
    nodes in N(e0) that are neighbours of e1.
    '''

    possible_nodes = new_nodes
    possible_nodes.extend([n for n in nx.all_neighbors(
        ego_net, e0) if ego_net.has_edge(n, e1)])
    possible_edges = get_unique_edges(powerset_of_2(possible_nodes))

    # Filter self-looping edges
    possible_edges = [(u, v) for u, v in possible_edges if u != v]

    return possible_edges


def set_new_edges(e0, e1, ego_net: nx.Graph, possible_edges, d, rho):
    '''
    The number of these edges is given by the
    required density of N(e1), i.e. d * (d - 1) * rho/2 - |E(N(e1))|

    where |E(N(e1))| is the number
    of edges between the neighbours of e1, which initially lie completely in N (e0).
    '''

    neighbors_of_e0 = list(nx.all_neighbors(ego_net, e0))
    neighbors_of_e1 = list(nx.all_neighbors(ego_net, e1))

    ENe1 = sum([list(ego_net.edges(n))
               for n in neighbors_of_e1], [])

    # Remove duplicate edges e.g. (2, 0) and (0,2) are the same
    ENe1 = get_unique_edges(ENe1)

    # Keep only those edges completely within neighbours of e0
    ENe1 = [(u, v)
            for u, v in ENe1 if (u in neighbors_of_e0) and (v in neighbors_of_e0)]

    num_set_edges = clamp(
        int(d * (d-1) * rho // 2 - len(ENe1)), 0, len(possible_edges))

    for u, v in random.sample(possible_edges, num_set_edges):
        ego_net.add_edge(u, v)


def get_incomplete_nodes(ego_net, d):
    incomplete_nodes = [n for n in ego_net.nodes() if ego_net.degree[n] < d]
    return incomplete_nodes


def sample_new_ego(ego_net, d):
    '''
    Get a random ego node that has not yet reached the defined degree of "d"
    '''
    # The nodes that have not reached the defined degree yet
    incomplete_nodes = get_incomplete_nodes(ego_net, d)
    # The ego node of the current iteration
    e1 = random.choice(incomplete_nodes)
    return e1


def create_new_nodes(ego_net, num_missing_nodes, e1):
    new_nodes = []

    for _ in range(num_missing_nodes):
        new_node = ego_net.number_of_nodes()
        ego_net.add_node(new_node)
        ego_net.add_edge(new_node, e1)
        new_nodes.extend([new_node])

    return new_nodes


def get_qualified_nodes(ego_net, d, e1):
    unqualified_nodes = list(set(sum([list(nx.all_neighbors(
        ego_net, neighbor)) for neighbor in nx.all_neighbors(ego_net, e1)], [])))

    qualified_nodes = [n for n in ego_net.nodes() if
                       (n not in unqualified_nodes) and (ego_net.degree(n) < d)]

    return qualified_nodes


def show_network(ego_net):
    # Seed layout for reproducibility
    pos = nx.spring_layout(ego_net)
    # Draw graph
    nx.draw(ego_net, pos, node_color="black", node_size=0.1, with_labels=False)


## Network generation config

In [4]:
d = 20
rho = config.average_clustering_coefficient
p = 0.85
seed = random.randrange(1,1000)
seed = 970


increase_steps = 200
decrease_steps = 100


## Network generation process

In [5]:

'''
The general procedure is to start with a random ego-network N (e0 ) with d and ρ taken
from the probability distributions (the “seed network”). 
'''



G = nx.erdos_renyi_graph(d, p, seed=seed)
# find node with largest degree
e0 = get_largest_hub(G)

ego_net: nx.Graph = nx.ego_graph(G, e0)

'''
with all nodes additionally
linked to e0.
'''
for n in ego_net.nodes():
    if n != e0:
        ego_net.add_edge(n, e0)
        

mapping = {old: new for old, new in zip(
    ego_net.nodes(), range(ego_net.number_of_nodes()))}
ego_net = nx.relabel_nodes(ego_net, mapping)

'''
The next ego-network N (e1) is created by picking an e1 ∈ N (e0) at
random, checking the number of missing edges d − d(e1) and adding as many new nodes
from scratch. These then have to be linked at random with each other and with those
nodes in N (e0 ) that are neighbours of e1.
'''
get_network_stats(ego_net)



  number of nodes: 19,                         number of edges: 141,                         average clustering coefficient: 0.8305597579425114,                         reciprocity: 0.0,                         average degree: 14.842105263157896                         diameter: 2,                         average path length: 1.1754385964912282                         average in degree: None                      


### 1. Increase step

In [6]:


step = 0
while step < increase_steps:

    e1 = sample_new_ego(ego_net, d)

    num_missing_nodes = d - ego_net.degree[e1]

    new_nodes = create_new_nodes(ego_net, num_missing_nodes, e1)

    possible_edges = get_possible_edges(e0, e1, ego_net, new_nodes)
    set_new_edges(e0, e1, ego_net, possible_edges, d, rho)

    e0 = e1
    step += 1



get_network_stats(ego_net)

  number of nodes: 1941,                         number of edges: 11118,                         average clustering coefficient: 0.8134013706793802,                         reciprocity: 0.0,                         average degree: 11.45595054095827                         diameter: 18,                         average path length: 8.939940088274192                         average in degree: None                      


### 2. Decrease step

In [7]:
step = 0
while step < decrease_steps:
    e1 = sample_new_ego(ego_net, d)

    num_missing_nodes = d - ego_net.degree[e1]

    '''
    During the decrease stage, we reduce the amount of new nodes, 
    while taking the rest from the exisiting nodes, conforming to 2 restrictions
    '''
    num_new_nodes = round((1 - step/decrease_steps) * num_missing_nodes)


    
    
    # The nodes newly created combined with the existing nodes qualified
    new_nodes = create_new_nodes(ego_net, num_new_nodes, e1)

    '''
    ...But only those are allowed which fulfill the requirements: (i) the degree d(ei)
    must be smaller than d, and (ii) they must not be in one of the ego-networks that the
    current ego-node ej is already in
    
    How do we find out that the existing node is not inside any of the network of the
    current ego node?
    
    1. find all the networks that e1 is in:
        follow up: How do we know that? 
    2. excluding that from the exisiting nodes list
    
    I mean the definition of an ego network is the central node and its direct neighbors right?
    So we just have to filter out all the ego networks formed by e1's direct neighbors, since any
    thing beyond that is not going to form a large enough ego network that would overlap e1   
    '''

    qualified_nodes = get_qualified_nodes(ego_net, d, e1)
    
    num_sample_nodes = clamp(num_missing_nodes - num_new_nodes, 0, len(qualified_nodes))
    
    new_nodes.extend(random.sample(qualified_nodes, num_sample_nodes))

    
    possible_edges = get_possible_edges(e0, e1, ego_net, new_nodes)
    set_new_edges(e0, e1, ego_net, possible_edges, d, rho)

    e0 = e1
    step += 1

get_network_stats(ego_net)

  number of nodes: 2395,                         number of edges: 15692,                         average clustering coefficient: 0.6622365086233541,                         reciprocity: 0.0,                         average degree: 13.103966597077244                         diameter: 10,                         average path length: 4.577209202547078                         average in degree: None                      


### 3. Completion step

In [8]:
skipped_nodes = []

while incomplete_nodes := [n for n in get_incomplete_nodes(ego_net, d) if (n not in skipped_nodes)]:
    e1 = random.choice(incomplete_nodes)
    
    qualified_nodes = get_qualified_nodes(ego_net,d,e1)
    
    if not qualified_nodes:
        skipped_nodes.extend([e1])
        continue
    
    num_missing_nodes = clamp(d - ego_net.degree[e1], 0, len(qualified_nodes))
    new_nodes = random.sample(qualified_nodes, num_missing_nodes)
    possible_edges = get_possible_edges(e0, e1, ego_net, new_nodes)
    
    if not possible_edges:
        skipped_nodes.extend([e1])
        continue
    
    set_new_edges(e0, e1, ego_net, possible_edges, d, rho)
    e0 = e1
    
    if (nx.transitivity(ego_net)) <= rho:
        break



get_network_stats(ego_net)

  number of nodes: 2395,                         number of edges: 18216,                         average clustering coefficient: 0.5627124843647432,                         reciprocity: 0.0,                         average degree: 15.211691022964509                         diameter: 7,                         average path length: 3.8274621836428233                         average in degree: None                      


In [9]:
ego_net = ego_net.to_directed()

modify_reciprocity(ego_net)

with open('data/ego_net.pkl', 'wb') as fp:
    pickle.dump(ego_net, fp)

network_to_csv(ego_net)

In [10]:

get_network_stats(ego_net)



  number of nodes: 2395,                         number of edges: 21860,                         average clustering coefficient: 0.5627124843647432,                         reciprocity: 0.3333943275388838,                         average degree: 18.254697286012526                         diameter: 11,                         average path length: 3.8274621836428233                         average in degree: 9.127348643006263                      
